In [ ]:
%matplotlib inline
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import utils, time #helper libraries
import matplotlib.pyplot as plt
import pandas as pd

# Stock Price Prediction

Beaucoup de méthodes existent déjà pour prédire l'évolution de valeurs sur un marché. Nous allons voir dans la suite de cet atelier un *réseau de neurones récurrents* ou _RNN_.

![caption](rnn.png)

Ces réseaux de neurones ont la particularité de pouvoir faire des prédictions sur des _séries_ ou _données séquentielles_. Ils sont dotés d'une mémoire qui leur permet de prendre en contact des données qui précèdent les données sur lesquels il travaille.

## Observation des données

Avant toutes choses, il est important de prendre connaissance des données avec lesquels nous allons entraîner notre algorithme. Nous utiliserons des relevés de l'indice boursier S&P 500 basé sur 500 grandes sociétés cotées sur les bourses américaines (Standard & Poor's).

La plupart des modèles développés aujourd'hui en _deep learning_ sont des modèles supervisés. C'est à dire qu'ils se "nourrissent" de données labellisées pour apprendre. Cela demande donc une excellente expertise de ces dernières pour que l'IA atteigne son objectif.

In [ ]:
# Nous chargeons les données
data = pd.read_csv('datas/sp500.csv', header=None)
# Affichons les 5 premières données
data.head()

In [ ]:
# Affichons les données
data.plot(legend=False)

In [ ]:
# Un peu de magie pour charger les données et normaliser
X_train, y_train, X_test, y_test = utils.load_data('datas/sp500.csv', 50, True)

## RNN network

Une fois que nous avons nos données prêtes, il nous faut un modèle. Nous avons ci-dessus un modèle comme ceci :

LSTM --> Dropout --> LSTM ---> Dropout --> Dense --> Linear Activation

In [ ]:
# Définition du model couche par couche
model = Sequential()

model.add(LSTM(
    input_dim=1,
    output_dim=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    100,
    return_sequences=False))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

model.compile(loss='mse', optimizer='rmsprop')

## Training

Une fois le modèle défini, il est temps de l'entraîner sur les données vues plus haut :

In [ ]:
# Nous pouvons jouer sur les paramêtres
history = model.fit(
                X_train,
                y_train,
                batch_size=512,
                nb_epoch=10,
                validation_split=0.05)

In [ ]:
# Affichons la "loss" et la "validation loss"
utils.plot_losses(history)

## Prédictions

Maintenant que le modèle est entraîné, regardons ce qu'il est capable de prédire sur de nouvelles données qu'il n'a jamais vu :

In [ ]:
# Le modèle prédit une seule valeur après une séquence
p = model.predict(X_test)    
plt.plot(y_test,color='blue', label='y_test')
plt.plot(p,color='yellow', label='prediction')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Le modèle prédit une suite de valeur pour avoir une "tendance"
predictions = utils.predict_sequences_multiple(model, X_test, 50, 50)
utils.plot_results_multiple(predictions, y_test, 50)

__Comment pouvons-nous améliorer les prédictions ?__